In [ ]:
#!pip install streamlit

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 534.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [2]:
%%writefile app.py

import copy
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 定义三个公园的 OLS 系数字典
PARK_COEFS = {
    "Flushing Meadow Park": {
        'Intercept': 495.5931,
        'Morning': -777.8152,
        'Evening': 174.6365,
        'Cloudy': -254.2291,
        'Rainy': -527.3739,
        'Extreme': -480.1169,
        'Hot': -277.0081,
        'Average': -77.6926,
        'Summer': 221.0166,
        'Weekend': 816.2640,
        'Shaded_Area_Minimal': -317.6069,
        'Windy': -466.3650,
        'non_water_area': 1102.1972,
        'Large_Event': 15710.0,
        'Small_Event': 419.7690
    },
    "Prospect Park": {
        'Intercept': 677.1632,
        'Morning': -472.8822,
        'Evening': -55.9915,
        'Cloudy': -69.3456,
        'Rainy': -266.9379,
        'Extreme': -298.9178,
        'Hot': -171.0609,
        'Cool': -171.6414,
        'Summer': 155.2552,
        'Weekend': 462.1574,
        'Shaded_Area_Partial': -82.5112,
        'Shaded_Area_Fully': -432.3021,
        'Windy': -63.1687,
        'non_water_area': 294.6552,
        'Large_Event': 1897.9544,
        'Small_Event': 1315.9990,
    },
    "Kissena Corridor Park": {
        'Intercept': 123.5902,
        'Morning': -91.7825,
        'Evening': 16.2103,
        'Cloudy': -24.9198,
        'Rainy': -134.6651,
        'Extreme': -120.6213,
        'Hot': -53.7868,
        'Average': 7.8254,
        'Summer': 9.2264,
        'Weekend': 62.1802,
        'Shaded_Area_Minimal': 289.1490,
        'Windy': -30.4440
    }
}

# 在文件开头或合适位置定义各公园的信息
PARK_INFO = {
    "Flushing Meadow Park": {
        "Intercept": 495.59,
        "Intercept Condition": [
            'non_summer', 'Weekday', 'Afternoon', 'Cool',
            'Clear', 'non_windy', 'Shaded_Area_Partial',
            'non_event', 'Water_Area'
        ],
        "Average visitor number": 420.37
    },
    "Prospect Park": {
        "Intercept": 677.16,
        "Intercept Condition": [
            'non_summer', 'Weekday', 'Afternoon', 'Average',
            'Clear', 'non_windy', 'Shaded_Area_Minimal',
            'non_event', 'Water_Area'
        ],
        "Average visitor number": 579.49
    },
    "Kissena Corridor Park": {
        "Intercept": 123.59,
        "Intercept Condition": [
            'non_summer', 'Weekday', 'Afternoon', 'Cool',
            'Clear', 'non_windy', 'Shaded_Area_Fully'
        ],
        "Average visitor number": 125.31
    }
}



#Prospect Park 交互项
INTERACTION_COEFS = {
    "Prospect Park": {
        'Cool:Shaded_Area_Partial': 58.3406,
        'Hot:Shaded_Area_Fully': 114.8829,
        'Cool:Shaded_Area_Fully': 119.2080,
        'Hot:non_water_area': -147.3022,
        'Cool:non_water_area': -48.1817
    }
    # 如果其他公园也有交互项，可以同理添加
}


# 2. 定义 feature 分组
FEATURE_GROUPS = {
    "Event": ["Large_Event", "Small_Event", "non_event"],
    "Time": ["Morning", "Afternoon", "Evening"],
    "Week": ["Weekend", "Weekday"],
    "Season": ["Summer", "non_summer"],
    "Temperature": ["Extreme", "Hot", "Average", "Cool"],
    "Weather": ["Clear", "Cloudy", "Rainy"],
    "Wind Condition": ["Windy", "non_windy"],
    "Water": ["Water_Area", "non_water_area"],
    "Shaded": ["Shaded_Area_Minimal", "Shaded_Area_Partial", "Shaded_Area_Fully"]
}

st.title("Park Visitor Number Forecast Dashboard")


# 1. 选择公园
st.write("### Park：")

park = st.radio(" ", list(PARK_COEFS.keys()), key="park")

st.write("### Feature：")

# 2. 根据 park 拷贝并修改分组
groups = {k: v.copy() for k, v in FEATURE_GROUPS.items()}

if park == "Flushing Meadow Park":
    groups["Shaded"].remove("Shaded_Area_Fully")
elif park == "Kissena Corridor Park":
    groups["Shaded"].remove("Shaded_Area_Partial")
    groups.pop("Water", None)
    groups.pop("Event", None)

# 按你想要的 3×3 排列顺序
layout_order = [
    "Season","Week","Time",
    "Temperature","Weather","Wind Condition",
    "Shaded","Event","Water"
]
# 只保留实际还在 groups 里的分类
layout_order = [g for g in layout_order if g in groups]

# 3. 渲染这一份特征单选框
selected_feats = []
for i in range(0, len(layout_order), 3):
    cols = st.columns(3)
    for col, group_name in zip(cols, layout_order[i:i+3]):
        with col:
            # 给 key 加上 park 前缀，确保每次都唯一
            key = f"{park}_{group_name}"
            choice = st.radio(group_name, groups[group_name], key=key)
            selected_feats.append(choice)

# 4. 点击按钮计算并存储
def run_prediction():
    coefs = PARK_COEFS[st.session_state.park]
    pred = coefs.get("Intercept", 0.0)
    chosen = []

    # 累加主效应
    for feat in selected_feats:
        chosen.append(feat)
        pred += coefs.get(feat, 0.0)

    # 如果是有交互项的公园，检查并累加交互效应
    park = st.session_state.park
    if park in INTERACTION_COEFS:
        for inter, beta in INTERACTION_COEFS[park].items():
            f1, f2 = inter.split(":")
            if f1 in chosen and f2 in chosen:
                pred += beta
                # 可选：把互动项名字也记录下来
                chosen.append(inter)

    st.session_state.pred = pred
    st.session_state.chosen_feats = chosen

# 按钮绑定
st.button("Submit", on_click=run_prediction)

# 显示结果（同前）
if "pred" in st.session_state:
    st.markdown("---")
    st.subheader("Result")


    # ① 基线信息（根据选中的公园展示）
    info = PARK_INFO[st.session_state.park]
    st.write(f"- **Park**：**{st.session_state.park}**")

    st.write(f"- **Intercept Condition:** {info['Intercept Condition']}")
    st.write(f"- **Intercept:** {info['Intercept']}")
    st.write(f"- **Average Visitor Number in Intercept Condition:** {info['Average visitor number']}")


    st.write(f"- **Feature Selected**：{st.session_state.chosen_feats}")
    #st.write(f"- **Visitor Number**：**{st.session_state.pred:.0f}**")
    st.markdown(
        f"- **Visitor Number**：**<span style='color:red'>{st.session_state.pred:.0f}</span>**",
        unsafe_allow_html=True
    )

Writing app.py


In [3]:
!nohup streamlit run app.py --server.port 8501 &

nohup: appending output to 'nohup.out'


In [4]:
import pyngrok
from pyngrok import ngrok

# 替换成你的实际 Authtoken
NGROK_AUTH_TOKEN = "2vKFTluwGhqgIsyf2bkFmbsN5bm_7EhqqhPiRAbqqYR1qduoj"  # 例如 "2FzA...b9jQ"

# 设置认证令牌
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 暴露 Streamlit 端口（默认 8501）
public_url = ngrok.connect(addr=8501, bind_tls=True)
print("公网访问地址:", public_url)

公网访问地址: NgrokTunnel: "https://91df-35-204-132-84.ngrok-free.app" -> "http://localhost:8501"
